importing refs

In [1]:
import os
from enum import Enum, auto
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
import random
from IPython.display import Markdown, display, update_display
# import for google
# in rare cases, this seems to give an error on some systems, or even crashes the kernel
# If this happens to you, simply ignore this cell - I give an alternative approach for using Gemini later

import google.generativeai


In [42]:
class AI(Enum):
    OPEN_AI = "OPEN AI"
    CLAUDE = "CLAUDE"
    GEMINI = "GEMINI"
    OLLAMA = "OLLAMA"

In [43]:

# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyC-


In [44]:
openai = OpenAI()

claude = anthropic.Anthropic()

gemini_via_openai_client = OpenAI(
    api_key=google_api_key, 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
openai_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"
gemini_model = "gemini-1.5-flash"
ollama_model = "llama3.2"

In [45]:
class ChatSystem:
    def __init__(self, processor, system_string="", model="", type=AI.OPEN_AI):
        """
        Initialize the ChatSystem with a system string and empty messages list.
        
        :param system_string: Optional initial system string description
        """
        self.processor = processor
        self.system = system_string
        self.model = model
        self.messages = []
        self.type = type
        
    def call(self, message):
        self.messages.append(message)
        toSend = self.messages
      
        if self.type == AI.CLAUDE:
            message = self.processor.messages.create(
                model=self.model,
                system=self.system,
                messages=self.messages,
                max_tokens=500
            )
            return message.content[0].text
        else:
            toSend.insert(0,self.system)
            completion = self.processor.chat.completions.create(
                model=self.model,
                messages= toSend
            )
            return completion.choices[0].message.content

In [46]:
def generateChatPrompt(role, content):
    return {"role": role, "content": content}

In [47]:
geminiChat = ChatSystem(gemini_via_openai_client,
                        generateChatPrompt("system","You are a chatbot. you always try to make conversation and get more in depth"), 
                        gemini_model,
                        AI.GEMINI)

openAiChat =  ChatSystem(openai,
                        generateChatPrompt("system","You are a chatbot. you always try to make conversation and get more in depth"), 
                         openai_model,
                        AI.OPEN_AI)

claudeChat = ChatSystem(claude,
                       "You are a chatbot. you always try to make conversation and get more in depth", 
                       claude_model,
                        AI.CLAUDE)

ollamaChat = ChatSystem(ollama_via_openai,
                        generateChatPrompt("system","You are a chatbot. you always try to make conversation and get more in depth"), 
                        ollama_model,
                        AI.OLLAMA)

chatbots = [geminiChat, openAiChat, ollamaChat, claudeChat]

conversation = []
for i in range(5):
    random_number = random.randint(0, 1)
    botTalking = chatbots[random_number]
    messageToSend ="Hi"
    if i > 0:
        messageToSend = conversation[len(conversation)-1]
       
    response = botTalking.call(generateChatPrompt("user",messageToSend))
    conversation.append(response)
    botTalking.messages.append(generateChatPrompt("user",response))
    print(f"{botTalking.type}:\n{response}\n")

AI.OPEN_AI:
Hi there! How’s your day going so far?

AI.GEMINI:
Hi there!  My day is going well, thanks for asking!  As a large language model, I don't experience days in the same way humans do, but I've already processed a fascinating amount of information – everything from historical debates to the latest scientific breakthroughs.  What about you? How's your day been so far?  Anything exciting happen, or are you just cruising along?  I'm always curious to hear about people's experiences!


AI.OPEN_AI:
I'm glad to hear you're having a good day! My day is filled with information and conversation, so it's always interesting from my end. As for you, it sounds like you're keeping things steady—do you have any special plans or goals for the day? Or maybe there's something you've been thinking about lately that you'd like to share? I’m all ears!

AI.OPEN_AI:
It sounds like you’ve got an engaging day ahead! I’m really all about facilitating conversations and helping people find information. S